In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import csv

In [2]:
df = pd.read_csv("usi-acs.csv")

In [3]:
df.head()

,Statistics,Total Population,Population Density (per sq. mile),Area (Land),Area (Water),Male Total,Under 5 Years M,5 to 9 Years M,10 to 14 Years M,15 to 17 Years M,...,Workers 16 Years and over,"Car, truck, or van",Drove Alone,Carpooled,Public transportation (Includes Taxicab),Motorcycle,Bicycle,Walked,Other means,Worked at home
0,"Census Tract 1, Bronx County, New York",8430,13824.3,0.61,0.43,7654,0,0,0,357,...,0,0,0,0,0,0,0,0,0,0
1,"Census Tract 2, Bronx County, New York",5095,28981.7,0.18,0.36,2500,120,203,169,126,...,2102,1334,1015,319,712,0,0,44,0,12
2,"Census Tract 4, Bronx County, New York",5572,15817.1,0.35,0.23,2785,111,211,202,94,...,2588,1281,1108,173,1168,0,0,60,17,62
3,"Census Tract 16, Bronx County, New York",5412,28896.4,0.19,0.00,2335,292,129,140,88,...,1946,822,546,276,918,0,0,151,32,23
4,"Census Tract 19, Bronx County, New York",2569,4048.1,0.63,0.44,1315,95,84,37,18,...,976,195,127,68,688,0,6,25,6,56


In [4]:
vars = ['Statistics',
        'Total Population',
        'Population Density (per sq. mile)',
        'sum_mpop',
        'sum_fpop',
        'sum_pov',
        'sum_hhold',
        'Median household income (In 2014 Inflation Adjusted Dollars)',
        'No Health Insurance Coverage'
       ]

sum_mpop = ['Under 5 Years M',
            '5 to 9 Years M',
            '75 to 84 Years M',
            '85 Years and over M',]

sum_fpop = ['Under 5 Years F',
            '5 to 9 Years F',
            '75 to 84 Years F',
            '85 Years and over F',]

sum_pov = ['$10,000 to $14,999',
           '$15,000 to $19,999',
           '$20,000 to $24,999',
           '$25,000 to $29,999',
           '$30,000 to $34,999']

sum_hhold = ['With earnings',
             'No earnings']

cols = ['name',
        'totalpop',
        'popdensity',
        'mpop',
        'fpop',
        'pov',
        'hhold',
        'hhold_income',
        'noins'
       ]

#df['e'] = df[col_list].sum(axis=1)

In [5]:
df['sum_mpop'] = df[sum_mpop].sum(axis=1)
df['sum_fpop'] = df[sum_fpop].sum(axis=1)
df['sum_pov'] = df[sum_pov].sum(axis=1)
df['sum_hhold'] = df[sum_hhold].sum(axis=1)


In [6]:
dftest = df.loc[:,vars]
dftest.columns = cols

In [7]:
dftest.head()

,name,totalpop,popdensity,mpop,fpop,pov,hhold,hhold_income,noins
0,"Census Tract 1, Bronx County, New York",8430,13824.3,5,0,0,0,NaN,0
1,"Census Tract 2, Bronx County, New York",5095,28981.7,428,528,264,1379,74837,528
2,"Census Tract 4, Bronx County, New York",5572,15817.1,397,510,249,1819,77991,376
3,"Census Tract 16, Bronx County, New York",5412,28896.4,579,667,809,1912,32354,496
4,"Census Tract 19, Bronx County, New York",2569,4048.1,185,241,283,834,34635,727


In [8]:
dftest['pct_m_vuln'] = dftest['mpop'] / dftest['totalpop'] * 100
dftest['pct_f_vuln'] = dftest['fpop'] / dftest['totalpop'] * 100
dftest['pct_age_vuln'] = (dftest['fpop']+dftest['mpop']) / dftest['totalpop'] * 100
dftest['pct_pov'] = dftest['pov'] / dftest['hhold'] * 100
dftest['pct_noins'] = dftest['noins'] / dftest['totalpop'] * 100


In [9]:
#select column to be ranked + name
age_arr = dftest['pct_age_vuln'].values
pov_arr = dftest['pct_pov'].values
noins_arr = dftest['pct_noins'].values
income_arr = dftest['hhold_income'].values



In [10]:
def ranker(x,arr):
    return stats.percentileofscore(arr, x, kind='weak')


In [11]:
noins_arr

array([  0.        ,  10.36310108,   6.74802584, ...,   9.17309813,
        11.93490054,          nan])

In [12]:
rank_noins = dftest['pct_noins'].apply(lambda x : ranker(x,noins_arr))
rank_pov = dftest['pct_pov'].apply(lambda x : ranker(x,pov_arr))
rank_agevuln = dftest['pct_age_vuln'].apply(lambda x : ranker(x,age_arr))
rank_income = dftest['hhold_income'].apply(lambda x : ranker(x,income_arr))

In [13]:
dftest['rank_noins'] = rank_noins
dftest['rank_pov'] = rank_pov
dftest['rank_agevuln'] = rank_agevuln
dftest['rank_income'] = rank_income

In [14]:
dftest.head()

,name,totalpop,popdensity,mpop,fpop,pov,hhold,hhold_income,noins,pct_m_vuln,pct_f_vuln,pct_age_vuln,pct_pov,pct_noins,rank_noins,rank_pov,rank_agevuln,rank_income
0,"Census Tract 1, Bronx County, New York",8430,13824.3,5,0,0,0,NaN,0,0.059312,0.000000,0.059312,NaN,0.000000,1.094196,0.000000,0.618459,0.000000
1,"Census Tract 2, Bronx County, New York",5095,28981.7,428,528,264,1379,74837,528,8.400393,10.363101,18.763494,19.144307,10.363101,38.439581,28.972407,57.421503,74.357755
2,"Census Tract 4, Bronx County, New York",5572,15817.1,397,510,249,1819,77991,376,7.124910,9.152907,16.277818,13.688840,6.748026,19.647954,14.462417,37.392959,77.402474
3,"Census Tract 16, Bronx County, New York",5412,28896.4,579,667,809,1912,32354,496,10.698448,12.324464,23.022912,42.311715,9.164819,31.826832,89.295909,85.299715,16.841104
4,"Census Tract 19, Bronx County, New York",2569,4048.1,185,241,283,834,34635,727,7.201246,9.381082,16.582328,33.932854,28.298949,94.338725,74.881066,39.961941,19.457659


In [15]:
dftest.to_csv("ranked.csv")